In [1]:
import numpy as np
import cv2 as cv
import trimesh
import matplotlib.pyplot as plt
import os
import json

from mesh_utils import slice_mesh_with_fuse
from visualisation import visualize_results_rays, visualize_projected, visualize_results_mesh

import cv2 as cv

In [2]:
IMG_FOLDER_PATH = 'image_selection_data/P_1'
IMG_FORMAT = '.JPG'
POSES_FOLDER = 'poses'
MESH_PATH = 'image_selection_data/cut_decimated_centered_textured_mesh.obj'

In [3]:
def get_camera_info(camera_id, camera_info_path = "cameras.json"):
    assert os.path.exists(camera_info_path), "Camera file not found" 
    with open(camera_info_path) as json_file:
        cameras_info = json.load(json_file)

    for i in cameras_info:
        if cameras_info[i]["id"] == camera_id:
            camera_info = cameras_info[i]
        break
    return camera_info


def load_image_info(image_name):
    image_path = os.path.join(IMG_FOLDER_PATH, image_name + IMG_FORMAT) 
    assert os.path.exists(image_path), "Image not found" 

    pose_path = os.path.join(POSES_FOLDER, image_name + '.json') 
    assert os.path.exists(pose_path), "Pose file not found" 
    with open(pose_path) as json_file:
        pose_info = json.load(json_file)

    camera_info = get_camera_info(pose_info["camera_id"]) 
    return pose_info, camera_info


In [13]:
image_name = 'DJI_20240419122022_0119_Z'
pose_info, camera_info = load_image_info(image_name)

In [14]:
def load_and_prepare_mesh(mesh_path, camera_matrix, rotation, center, img_height, img_width, vis=False):
    pier = trimesh.load(mesh_path, force='mesh')
    pier_cutted = slice_mesh_with_fuse(rotation, center, camera_matrix/10, int(img_height*2), int(img_width*2), pier)
    if vis:     
        meshes = []
        world_xyz = trimesh.creation.axis()
        meshes.append(world_xyz)
        camera_xyz = trimesh.creation.axis()
        camera_xyz.vertices = (np.matmul(rotation.transpose(), camera_xyz.vertices.transpose()) + center).transpose()
        meshes.append(camera_xyz)
        meshes.append(pier_cutted)
        scene = trimesh.Scene(meshes)
        scene.show('notebook')
        
    return pier_cutted

def prepare_camera_and_pose_data(camera_info, pose_info):
    camera_matrix = np.float64(camera_info["matrix"])
    distortion_coefficients = np.float64(camera_info["distortion_coefficients"])
    rotation = np.float64(pose_info["rotation"]).reshape(3, 3)
    center = np.float64(pose_info["center"]).reshape(3, 1)
    img_height = camera_info["height"]
    img_width = camera_info["width"]
    return camera_matrix, distortion_coefficients, rotation, center, img_height, img_width



In [15]:
_camera_matrix, _distortion_coefficients, _rotation, _center, _img_height, _img_width = prepare_camera_and_pose_data(camera_info, pose_info)
_pier_cutted = load_and_prepare_mesh(MESH_PATH, _camera_matrix, _rotation, _center, _img_height, _img_width, vis=False)

In [16]:
def get_vertices_behind_camera(vertices, rotation, translation_cam):
    vertices = np.array(vertices, dtype=np.float64)
    rotation = np.array(rotation, dtype=np.float64)
    translation_cam = np.array(translation_cam, dtype=np.float64).squeeze()

    transform = np.eye(4, dtype=np.float64)
    transform[:3, :3] = rotation
    transform[:3, 3] = translation_cam
    homogeneous_vertices = np.hstack((vertices, np.ones((vertices.shape[0], 1))))
    camera_coords = np.dot(homogeneous_vertices, transform.T)
    behind_camera = camera_coords[:, 2] < 0

    return behind_camera


def project_vertices(mesh, rotation, center, camera_matrix, distortion_coefficients):
    translation_cam = (-rotation @ center).reshape(3, 1)
    projected_vertices, _ = cv.projectPoints(mesh.vertices.view(np.ndarray).astype(np.float64),
                                             np.float64(rotation), np.float64(translation_cam),
                                             np.float64(camera_matrix),
                                             np.float64(distortion_coefficients))
    projected_vertices = projected_vertices.squeeze() #.astype(np.int64)
    return projected_vertices

In [17]:
projected_vertices = project_vertices(_pier_cutted, _rotation, _center, _camera_matrix, _distortion_coefficients)

In [18]:
# visualize_projected(projected_vertices, (img_height, img_width))

In [19]:
def filter_vertices(vertices, projected_vertices, img_height, img_width, rotation, center):
    print(f"Total vertices: {len((vertices))}")
    print(f"Vertices in image coordinates shape: {projected_vertices.shape}")
    behind_camera = behind_camera = get_vertices_behind_camera(vertices, rotation, (-rotation @ center).reshape(3, 1))
    in_front_of_camera = ~behind_camera

    within_x_bounds = (projected_vertices[:, 0] >= 0) & (projected_vertices[:, 0] < img_width)
    within_y_bounds = (projected_vertices[:, 1] >= 0) & (projected_vertices[:, 1] < img_height)
    within_image_bounds = within_x_bounds & within_y_bounds
    potentially_visible = in_front_of_camera & within_image_bounds

    potentially_visible = within_image_bounds
    print(f"Vertices in front of camera: {np.sum(in_front_of_camera)}")
    print(f"Vertices within image boundaries: {np.sum(within_image_bounds)}")
    print(f"Potentially visible vertices: {np.sum(potentially_visible)}")
    potential_indices = np.where(potentially_visible)[0]      
    return potential_indices

def check_visibility(mesh, vertices, potential_indices, center, treshhold=0.01):

    print('Total vertices', len(vertices))
    vertices_projected = vertices[potential_indices]
    print('Projected vertices', len(vertices_projected))

    rays_directions = vertices_projected - center.squeeze()
    distances = np.linalg.norm(rays_directions, axis=1)
    rays_directions /= distances.reshape(-1, 1)
    rays_origins = np.tile(center.squeeze(), (len(rays_directions), 1))

    locations, index_ray, index_tri = mesh.ray.intersects_location(
        ray_origins=rays_origins,
        ray_directions=rays_directions,
        multiple_hits=False
    )

    any_hit = mesh.ray.intersects_any(
        ray_origins=rays_origins,
        ray_directions=rays_directions
    )

    potential_indices_hits = np.where(any_hit)[0]
    print('Ray hits',len(potential_indices_hits))

    vertices_projected_hits = vertices_projected[potential_indices_hits]

    ray_miss = np.abs(vertices_projected_hits - locations) # score to deside if vertice is visible
    ray_miss = ray_miss < treshhold

    visible = np.logical_and(ray_miss[:, 0], np.logical_and(ray_miss[:, 1], ray_miss[:, 2]))
    print('Visible', sum(visible))

    visible_indices = potential_indices[potential_indices_hits[visible]]
    visible_vertices = vertices[visible_indices]
    print('Visible vertices', len(visible_vertices))

    vis_rays_origins = rays_origins[potential_indices_hits[visible]]
    vis_rays_directions =  rays_directions[potential_indices_hits[visible]]
    return visible_indices, vis_rays_origins, vis_rays_directions

In [20]:
vertices = _pier_cutted.vertices.view(np.ndarray).astype(np.float64)

potential_indices = filter_vertices(vertices, projected_vertices, _img_height, _img_width, _rotation, _center)
visible_indices, vis_rays_origins, vis_rays_directions = check_visibility(_pier_cutted, vertices, potential_indices, _center, treshhold=0.01)
camera_position = _center.squeeze()

visualize_results_rays(_pier_cutted,
                      visible_indices,
                      _rotation,
                      _center,
                      vis_rays_origins,
                      vis_rays_directions)


Total vertices: 673
Vertices in image coordinates shape: (673, 2)
Vertices in front of camera: 673
Vertices within image boundaries: 359
Potentially visible vertices: 359
Total vertices 673
Projected vertices 359
Ray hits 348
Visible 80
Visible vertices 80


In [21]:
visualize_results_mesh(_pier_cutted, visible_indices, _rotation, _center, vis_rays_origins, vis_rays_directions)